In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
# parse confusion matrix
# Just to test the code, the code below was run on some older data

os.chdir('/home/hermes/radiology')
cwd = os.getcwd()
print(cwd)

/home/hermes/radiology


In [3]:
# Get list of all models and confusion matrix of each fold
sourcedir = 'RadiologyPaper_June2022//'
modelsdir = 'FinalModels_Dec2021_KFold'
folder = ''
models = os.listdir(modelsdir)
print(models)
models_nodir = []
directions = []
for m in models:
    # Get model name without left/right direction
    if('left' in m):
        model_nodir = m.replace('_left_','_')
        dirtemp = 'left'
    else:
        model_nodir = m.replace('_right_','_')
        dirtemp = 'right'
    models_nodir.append(model_nodir)
    directions.append(dirtemp)
    
print(models_nodir)

['model_RESNET_left_spine_double', 'model_RESNET_right_spine_double']
['model_RESNET_spine_double', 'model_RESNET_spine_double']


In [4]:
def getcm_num(cm0):
    cm = []
    for c1 in cm0:
        ctemp = []
        for c2 in c1:
            ctemp.append(int(c2))
        cm.append(ctemp)  
    return cm

In [5]:
def get_basic_acc(cm0):
    tp = []
    cm = []
    cm = getcm_num(cm0)
    tp.append(cm[0][0])
    tp.append(cm[1][1])
    tp.append(cm[2][2])
    tp.append(cm[3][3])
    totals = [cm[0][0]+cm[0][1]+cm[0][2]+cm[0][3],cm[1][0]+cm[1][1]+cm[1][2]+cm[1][3],cm[2][0]+cm[2][1]+cm[2][2]+cm[2][3], cm[3][0]+cm[3][1]+cm[3][2]+cm[3][3] ] # TODO - change for right !
    print('Total is ',totals)
    fn = []
    for ind, t in enumerate(totals):
        fn.append(totals[ind] -tp[ind])
    fp = []
    fp.append(cm[1][0]+cm[2][0]+cm[3][0])
    fp.append(cm[0][1]+cm[2][1]+cm[3][1])
    fp.append(cm[0][2]+cm[1][2]+cm[3][2])
    fp.append(cm[0][3]+cm[1][3]+cm[2][3])
    tn = []
    tn.append(cm[1][1]+cm[1][2]+cm[1][3]+cm[2][1]+cm[2][2]+cm[2][3]+cm[3][1]+cm[3][2]+cm[3][3])
    tn.append(cm[0][0]+cm[0][2]+cm[0][3]+cm[2][0]+cm[2][2]+cm[2][3]+cm[3][0]+cm[3][2]+cm[3][3])
    tn.append(cm[0][0]+cm[0][1]+cm[0][3]+cm[1][0]+cm[1][1]+cm[1][3]+cm[3][0]+cm[3][1]+cm[3][3])
    tn.append(cm[0][0]+cm[0][1]+cm[0][2]+cm[1][0]+cm[1][1]+cm[1][2]+cm[2][0]+cm[2][1]+cm[2][2])
    prec = []
    prec_total = 0
    for ind, t in enumerate(totals):    
        nr = tp[ind]
        dr = tp[ind]+fp[ind]
        prectemp = nr/(dr+0.0001)
        prec.append(prectemp)
        prec_total +=prectemp*totals[ind]
    prec_total = prec_total*100/sum(totals)
    rec = []
    rec_total = 0
    for ind, t in enumerate(totals):    
        nr = tp[ind]
        #dr = tp[ind]+fn[ind]
        dr = totals[ind]
        rectemp = nr/dr
        rec.append(rectemp)
        rec_total +=(rectemp*totals[ind])
    rec_total = rec_total*100/sum(totals)
    f1 = []
    f1_total = 0
    for ind, t in enumerate(totals):    
        f1temp = 2 * (prec[ind] * rec[ind]) / (prec[ind] + rec[ind]+0.0001)
        f1.append(f1temp)
        f1_total +=f1temp*totals[ind]
    f1_total = f1_total*100/sum(totals)
    print('Total recall is ',rec_total)
    print('Total precision is ',prec_total)
    print('f1 is ',f1)
    print('Total f1 is ',f1_total)
    return prec_total, rec_total, rec_total, f1_total

In [6]:
def get_binary_acc(cm0):
    # Matrix 1- No disease vs. ALL others
    cm = getcm_num(cm0)
    cm_othertn = cm[1][1] + cm[1][2] + cm[1][3]+cm[2][1] + cm[2][2] + cm[2][3]+cm[3][1] + cm[3][2] + cm[3][3]
    cm_fn = cm[1][0]+cm[2][0]+cm[3][0]
    cm_novsall = [[cm[0][0], cm[0][1]+cm[0][2]+cm[0][3]],[cm_fn, cm_othertn]]
    tn = cm_novsall[0][0]
    fp = cm_novsall[0][1]
    tp = cm_novsall[1][1]
    fn = cm_novsall[1][0]
    acc1 = (tp+tn)/(tp+fp+tn+fn)
    print('None vs. all Overall accuracy is ',acc1*100)
    recall1 = tp/(tp+fn)
    print('None vs. all  Recall is ',recall1*100)
    prec1 =tp/(tp+fp)
    print('None vs. all Precision is ',prec1*100)
    f11 =  2 * (prec1* recall1) / (prec1 + recall1)
    print('None vs. all  F1 is ',f11*100)
    totals1 = cm[0][0]+cm[0][1]+cm[0][2]+cm[0][3]
    totals2 = cm[1][0]+cm[1][1]+cm[1][2]+cm[1][3]
    totals3 = cm[2][0]+cm[2][1]+cm[2][2]+cm[2][3]
    totals4 = cm[3][0]+cm[3][1]+cm[3][2]+cm[3][3]
    cm_tn = cm[0][0]+cm[0][1]+cm[1][0]+cm[1][1]
    cm_tp = cm[2][2]+cm[2][3]+cm[3][2]+cm[3][3]
    cm_fp = cm[0][2]+cm[0][3]+cm[1][2]+cm[1][3]
    cm_fn = cm[2][0]+cm[2][1]+cm[3][0]+cm[3][1]
    cm_nonemildvsmedsev = [[cm_tn, cm_fp],[cm_fn, cm_tp]]
    print(cm_nonemildvsmedsev)
    acc2 = (cm_tp+cm_tn)/(cm_tp+cm_fp+cm_tn+cm_fn)
    print('None or Mild vs. med/Severe Overall accuracy is ',acc2*100)
    recall2 = cm_tp/(cm_tp+cm_fn)
    print('None or Mild vs. med/Severe  Recall is ',recall2*100)
    prec2=cm_tp/(cm_tp+cm_fp)
    print('None or Mild vs. med/Severe  Precision is ',prec2*100)
    f12 =  2 * (prec2* recall2) / (prec2 + recall2)
    print('None or Mild vs. med/Severe  F1 is ',f12*100)
    cm_tn = cm[0][0]+cm[0][1]+cm[0][2]+cm[1][0]+cm[1][1]+cm[1][2]+cm[2][0]+cm[2][1]+cm[2][2]
    cm_tp = cm[3][3]
    cm_fp = cm[0][3]+cm[1][3]+cm[2][3]
    cm_fn = cm[3][0]+cm[3][1]+cm[3][2]
    cm_nonemildmedvssev = [[cm_tn, cm_fp],[cm_fn, cm_tp]]
    print(cm_nonemildmedvssev)
    try:
        print('Inside try loop...')
        acc3 = (cm_tp+cm_tn)/(cm_tp+cm_fp+cm_tn+cm_fn)
        print('None or mild/medium vs. Highest Severity Overall accuracy is ',acc3*100)
        recall3 = cm_tp/(cm_tp+cm_fn)
        print('None or Mild/medium vs. Highest Severity  Recall is ',recall3*100)
        prec3 =cm_tp/(cm_tp+cm_fp)
        print('None or mild/medium vs. Highest Severity Precision is ',prec3*100)
        f13 =  2 * (prec3* recall3) / (prec3 + recall3)
        print('None or Mild/med vs. Highest Severity  F1 is ',f13*100)
    except:
        print('Inside except loop....')
        acc3 = (cm_tp+cm_tn)/(cm_tp+cm_fp+cm_tn+cm_fn+0.0001)
        print('None or mild/medium vs. Highest Severity Overall accuracy is ',acc3*100)
        recall3 = cm_tp/(cm_tp+cm_fn+0.0001)
        print('None or Mild/medium vs. Highest Severity  Recall is ',recall3*100)
        prec3 =cm_tp/(cm_tp+cm_fp+0.0001)
        print('None or mild/medium vs. Highest Severity Precision is ',prec3*100)
        f13 =  2 * (prec3* recall3) / (prec3 + recall3+0.0001)
        print('None or Mild/med vs. Highest Severity  F1 is ',f13*100)
        
    mae0 = (1/totals1)*(abs(0-0)*cm[0][0] + abs(0-1)*cm[0][1] + abs(0-2)*cm[0][2] + abs(0-3)*cm[0][3])
    mae1 = (1/totals2)*(abs(1-0)*cm[1][0] + abs(1-1)*cm[1][1] + abs(1-2)*cm[1][2] + abs(1-3)*cm[1][3])
    mae2 = (1/totals3)*(abs(2-0)*cm[2][0] + abs(2-1)*cm[2][1] + abs(2-2)*cm[2][2] + abs(2-3)*cm[2][3])
    mae3 = (1/totals4)*(abs(3-0)*cm[3][0] + abs(3-1)*cm[3][1] + abs(3-2)*cm[3][2] + abs(3-3)*cm[3][3])
    print('MAE 0' , mae0)
    print('MAE 1' , mae1)
    print('MAE 2' , mae2)
    print('MAE 3' , mae3)
    mae =  0.25*(mae0+mae1+mae2+mae3)
    print('Mean MAE' ,mae)
    return prec1*100, acc1*100, recall1*100, f11*100, prec2*100, acc2*100, recall2*100, f12*100, prec3*100, acc3*100, recall3*100, f13*100, mae

In [7]:
def get_metrics_string(prec, acc, recall, f1):
    prec = str(round(prec, 2))
    acc = str(round(acc, 2))
    recall = str(round(recall, 2))
    f1 = str(round(f1, 2))
    metrics_str = 'Precision- '+ prec +' ; Accuracy- '+ acc+' ; Recall- '+ recall +' ; F1- '+ f1
    print(metrics_str)
    return metrics_str

In [8]:
def parse_cm(cm):
    print(cm)
    cm = cm.replace('[','')
    cm = cm.replace(']','')
    # get each row
    cm0 = []
    for i in range(0,4):
        cmtemp = cm.split('\n')[i]
        cmtemp.replace('\n','')
        cm0.append(cmtemp)
    # get each element
    
    # Get final 2D matrix
    cmfinal = []
    for c in cm0:
        celements = c.split(' ')
        cnew = []
        for c0 in celements:
            c0.strip()
            if(len(c0)>=1):
                cnew.append(c0)
        cmfinal.append(cnew)
    print(cmfinal)
    
                    
    print('-'*80)
    return cmfinal

In [9]:
prec = []
acc = []
recall = []
f1 = []

prec1 = []
acc1 = []
recall1 = []
f11 = []

prec2 = []
acc2 = []
recall2 = []
f12 = []

prec3 = []
acc3 = []
recall3 = []
f13 = []

prec4 = []
acc4 = []
recall4 = []
f14 = []

maefinal = []

In [10]:
# For each model, get the confusion matrix
allmodels = []
allfolds = []
for m in models:
    print(m)
    subdirs = os.listdir(os.path.join(modelsdir,m))
    for s in subdirs:
        print(s)
        cm_file = os.path.join(modelsdir,m,s,'confusion_matrix.txt')        
        with open(cm_file) as f:
            contents = f.read()
        cm = parse_cm(contents)
        prectemp, acctemp, recalltemp, f1temp = get_basic_acc(cm)
        prec1temp, acc1temp, recall1temp, f11temp, prec2temp, acc2temp, recall2temp, \
        f12temp, prec3temp, acc3temp, recall3temp, f13temp, mae  = get_binary_acc(cm)
        metrics_string0 = get_metrics_string(prectemp, acctemp, recalltemp, f1temp)
        metrics_string1 = get_metrics_string(prec1temp, acc1temp, recall1temp, f11temp)
        metrics_string2 = get_metrics_string(prec2temp, acc2temp, recall2temp, f12temp)
        metrics_string3 = get_metrics_string(prec3temp, acc3temp, recall3temp, f13temp)
        # Update values for regular 4-class classification model
        prec.append(prectemp)
        acc.append(acctemp)
        recall.append(recalltemp)
        f1.append(f1temp)
        # Update values for binary classification model 1
        prec1.append(prec1temp)
        acc1.append(acc1temp)
        recall1.append(recall1temp)
        f11.append(f11temp)
        # Update values for binary classification model 2
        prec2.append(prec2temp)
        acc2.append(acc2temp)
        recall2.append(recall2temp)
        f12.append(f12temp)
        # Update values for binary classification model 3
        prec3.append(prec3temp)
        acc3.append(acc3temp)
        recall3.append(recall3temp)
        f13.append(f13temp)
        # Update values for MAE
        maefinal.append(mae)
        allmodels.append(m)
        allfolds.append(s)
        
        

model_RESNET_left_spine_double
Fold5
[[1006  221   22   12]
 [ 271  657  229   57]
 [  25  195  415  182]
 [   5   40  173  339]]

[['1006', '221', '22', '12'], ['271', '657', '229', '57'], ['25', '195', '415', '182'], ['5', '40', '173', '339']]
--------------------------------------------------------------------------------
Total is  [1261, 1214, 817, 557]
Total recall is  62.79553130683294
Total precision is  62.64927024658255
f1 is  [0.7834390852955537, 0.5646256222528921, 0.5011577130128005, 0.5910572303433786]
Total f1 is  62.66658666680758
None vs. all Overall accuracy is  85.55468952974799
None vs. all  Recall is  88.3693972179289
None vs. all Precision is  89.96852871754524
None vs. all  F1 is  89.16179337231969
[[2155, 320], [265, 1109]]
None or Mild vs. med/Severe Overall accuracy is  84.8012470771629
None or Mild vs. med/Severe  Recall is  80.71324599708879
None or Mild vs. med/Severe  Precision is  77.60671798460463
None or Mild vs. med/Severe  F1 is  79.12950410274705
[[30

[[1099  181    9    5]
 [ 256  551  200   22]
 [  34  176  422  124]
 [  10   39  180  300]]

[['1099', '181', '9', '5'], ['256', '551', '200', '22'], ['34', '176', '422', '124'], ['10', '39', '180', '300']]
--------------------------------------------------------------------------------
Total is  [1294, 1029, 756, 529]
Total recall is  65.74279379157429
Total precision is  65.42384139486727
f1 is  [0.8161401712966417, 0.557642370039828, 0.5385588384915533, 0.6121951562588038]
Total f1 is  65.43517461167846
None vs. all Overall accuracy is  86.28048780487805
None vs. all  Recall is  87.03543647363871
None vs. all Precision is  91.17247623358986
None vs. all  F1 is  89.0559363254477
[[2087, 236], [259, 1026]]
None or Mild vs. med/Severe Overall accuracy is  86.28048780487805
None or Mild vs. med/Severe  Recall is  79.8443579766537
None or Mild vs. med/Severe  Precision is  81.29952456418383
None or Mild vs. med/Severe  F1 is  80.56537102473499
[[2928, 151], [229, 300]]
Inside try loop..

In [11]:
# Get 1st table - all data all folds, all basic numbers + string
pd.DataFrame({'Model':allmodels,'Fold':allfolds,'Precision':prec,'Recall':recall,'F1':f1,
             'MAE':maefinal}).to_csv('TempResults_Aug2022_BestModels5fold_BasicStats_All.csv')

In [12]:
# Get 1st table - all data all folds, all basic numbers for binary classification
pd.DataFrame({'Model':allmodels,'Fold':allfolds,'Precision':prec,'Recall':recall,'F1':f1,
             'MAE':maefinal,'Prec_None_Vs_MildMedSevere':prec1,'Accuracy_None_Vs_MildMedSevere':acc1,
             'Recall_None_Vs_MildMedSevere':recall1,'F1_None_Vs_MildMedSevere':f11,
             'Prec_NoneMild_Vs_MedSevere':prec2,'Accuracy_NoneMild_Vs_MedSevere':acc2,
             'Recall_NoneMild_Vs_MedSevere':recall2,'F1_NoneMild_Vs_MedSevere':f12,
             'Prec_NoneMildMed_Vs_Severe':prec3,'Accuracy_NoneMildMed_Vs_Severe':acc3,
             'Recall_NoneMildMed_Vs_Severe':recall3,'F1_NoneMildMed_Vs_Severe':f13,}).to_csv('TempResults_Aug2022_BestModels5fold_AdvancedStats_All.csv')

In [13]:
# For each model, aggregate over all the folds
allmodels = []
allfolds = []
maemean = []
maestdev = []
precmean = []
precstdev = []
accmean = []
accstdev = []
recallmean = []
recallstdev = []
f1mean = []
f1stdev = []

prec1mean = []
prec1stdev = []
acc1mean = []
acc1stdev = []
recall1mean = []
recall1stdev = []
f11mean = []
f11stdev = []

prec2mean = []
prec2stdev = []
acc2mean = []
acc2stdev = []
recall2mean = []
recall2stdev = []
f12mean = []
f12stdev = []

prec3mean = []
prec3stdev = []
acc3mean = []
acc3stdev = []
recall3mean = []
recall3stdev = []
f13mean = []
f13stdev = []

modelsagg = []
for m in models:
    print(m)
    subdirs = os.listdir(os.path.join(modelsdir,m))
    precvals = []
    accvals = []
    recallvals = []
    f1vals = []
    maevals = []
    prec1vals = []
    acc1vals = []
    recall1vals = []
    f11vals = []
    prec2vals = []
    acc2vals = []
    recall2vals = []
    f12vals = []
    prec3vals = []
    acc3vals = []
    recall3vals = []
    f13vals = []
    
    
    for s in subdirs:
        print(s)
        cm_file = os.path.join(modelsdir,m,s,'confusion_matrix.txt')        
        with open(cm_file) as f:
            contents = f.read()
        cm = parse_cm(contents)
        prectemp, acctemp, recalltemp, f1temp = get_basic_acc(cm)
        prec1temp, acc1temp, recall1temp, f11temp, prec2temp, acc2temp, recall2temp, \
        f12temp, prec3temp, acc3temp, recall3temp, f13temp, mae  = get_binary_acc(cm)
        metrics_string0 = get_metrics_string(prectemp, acctemp, recalltemp, f1temp)
        metrics_string1 = get_metrics_string(prec1temp, acc1temp, recall1temp, f11temp)
        metrics_string2 = get_metrics_string(prec2temp, acc2temp, recall2temp, f12temp)
        metrics_string3 = get_metrics_string(prec3temp, acc3temp, recall3temp, f13temp)
        # Update values for regular 4-class classification model
        precvals.append(prectemp)
        accvals.append(acctemp)
        recallvals.append(recalltemp)
        f1vals.append(f1temp)
        # Update values for binary classification model 1
        prec1vals.append(prec1temp)
        acc1.append(acc1temp)
        recall1vals.append(recall1temp)
        f11vals.append(f11temp)
        
        # Update values for binary classification model 2
        prec2vals.append(prec2temp)
        acc2vals.append(acc2temp)
        recall2vals.append(recall2temp)
        f12vals.append(f12temp)
        # Update values for binary classification model 3
        prec3vals.append(prec3temp)
        acc3vals.append(acc3temp)
        recall3vals.append(recall3temp)
        f13vals.append(f13temp)
        # Update values for MAE
        maefinal.append(mae)
        maevals.append(mae)
        allmodels.append(m)
        allfolds.append(s)
    maestdev_temp = np.std(maevals)
    maemean_temp = np.mean(maevals)
    precstdev_temp = np.std(precvals)
    precmean_temp = np.mean(precvals)
    print('Mean precision is ',precmean_temp,' Stdev is ',precstdev_temp)
    recallstddev_temp = np.std(recallvals)
    recallmean_temp = np.mean(recallvals)
    print('Mean recall is ',precmean_temp,' Stdev is ',precstdev_temp)
    f1stddev_temp = np.std(f1vals)
    f1mean_temp = np.mean(f1vals)
    print('Mean f1 is ',f1mean_temp,' Stdev is ',f1stddev_temp)
    
    # same as above for the binary models 
    prec1stdev_temp = np.std(prec1vals)
    prec1mean_temp = np.mean(prec1vals)
    print('Mean precision is ',prec1mean_temp,' Stdev is ',prec1stdev_temp)
    recall1stddev_temp = np.std(recall1vals)
    recall1mean_temp = np.mean(recall1vals)
    print('Mean recall is ',recall1mean_temp,' Stdev is ',recall1stddev_temp)
    f11stddev_temp = np.std(f11vals)
    f11mean_temp = np.mean(f11vals)
    print('Mean f1 is ',f11mean_temp,' Stdev is ',f11stddev_temp)
    
    
        # same as above for the binary models 
    prec2stdev_temp = np.std(prec2vals)
    prec2mean_temp = np.mean(prec2vals)
    print('Mean precision is ',prec2mean_temp,' Stdev is ',prec2stdev_temp)
    recall2stddev_temp = np.std(recall2vals)
    recall2mean_temp = np.mean(recall2vals)
    print('Mean recall is ',recall2mean_temp,' Stdev is ',recall2stddev_temp)
    f12stddev_temp = np.std(f12vals)
    f12mean_temp = np.mean(f12vals)
    print('Mean f1 is ',f12mean_temp,' Stdev is ',f12stddev_temp)
    
        # same as above for the binary models 
    prec3stdev_temp = np.std(prec3vals)
    prec3mean_temp = np.mean(prec3vals)
    print('Mean precision is ',prec3mean_temp,' Stdev is ',prec3stdev_temp)
    recall3stddev_temp = np.std(recall3vals)
    recall3mean_temp = np.mean(recall3vals)
    print('Mean recall is ',recall3mean_temp,' Stdev is ',recall3stddev_temp)
    f13stddev_temp = np.std(f13vals)
    f13mean_temp = np.mean(f13vals)
    print('Mean f1 is ',f13mean_temp,' Stdev is ',f13stddev_temp)
    
    
    maemean.append(maemean_temp)
    maestdev.append(maestdev_temp)
    precmean.append(precmean_temp)
    precstdev.append(precstdev_temp)
    recallmean.append(recallstddev_temp)
    recallstdev.append(recallmean_temp)
    f1mean.append(f1stddev_temp)
    f1stdev.append(f1mean_temp)
    
    prec1mean.append(prec1mean_temp)
    prec1stdev.append(prec1stdev_temp)
    recall1mean.append(recall1stddev_temp)
    recall1stdev.append(recall1mean_temp)
    f11mean.append(f11stddev_temp)
    f11stdev.append(f11mean_temp)
    
    prec2mean.append(prec2mean_temp)
    prec2stdev.append(prec2stdev_temp)
    recall2mean.append(recall2stddev_temp)
    recall2stdev.append(recall2mean_temp)
    f12mean.append(f12stddev_temp)
    f12stdev.append(f12mean_temp)
    
    prec3mean.append(prec3mean_temp)
    prec3stdev.append(prec3stdev_temp)
    recall3mean.append(recall3stddev_temp)
    recall3stdev.append(recall3mean_temp)
    f13mean.append(f13stddev_temp)
    f13stdev.append(f13mean_temp)
    
    
    modelsagg.append(m)

model_RESNET_left_spine_double
Fold5
[[1006  221   22   12]
 [ 271  657  229   57]
 [  25  195  415  182]
 [   5   40  173  339]]

[['1006', '221', '22', '12'], ['271', '657', '229', '57'], ['25', '195', '415', '182'], ['5', '40', '173', '339']]
--------------------------------------------------------------------------------
Total is  [1261, 1214, 817, 557]
Total recall is  62.79553130683294
Total precision is  62.64927024658255
f1 is  [0.7834390852955537, 0.5646256222528921, 0.5011577130128005, 0.5910572303433786]
Total f1 is  62.66658666680758
None vs. all Overall accuracy is  85.55468952974799
None vs. all  Recall is  88.3693972179289
None vs. all Precision is  89.96852871754524
None vs. all  F1 is  89.16179337231969
[[2155, 320], [265, 1109]]
None or Mild vs. med/Severe Overall accuracy is  84.8012470771629
None or Mild vs. med/Severe  Recall is  80.71324599708879
None or Mild vs. med/Severe  Precision is  77.60671798460463
None or Mild vs. med/Severe  F1 is  79.12950410274705
[[30

In [14]:

print(len(modelsagg))
print(len(precmean))
print(len(precstdev))
print(len(recallmean))
print(len(recallstdev))
print(len(f1mean))
print(len(f1stdev))
print(len(maemean))
print(len(maestdev))



# Get 1st table - all data all folds, all basic numbers + string
pd.DataFrame({'Model':modelsagg,'Precision Mean':precmean,'Precision Stdev':precstdev,
              'Recall Mean ':recallmean,'Recall Stdev ':recallstdev,'F1 Mean':f1mean,'F1 stdev':f1stdev,
             'MAE Mean ': maemean, 'MAE Stdev ': maestdev}).to_csv('TempResults_Dec2022_BestModels5fold_Aggregate_BasicStats_All.csv')

2
2
2
2
2
2
2
2
2


In [15]:
pd.DataFrame({'Model':modelsagg,'Precision 1 Mean':prec1mean,'Precision 1 Stdev':prec1stdev,
              'Recall 1 Mean ':recall1mean,'Recall 1 Stdev ':recall1stdev,'F1 1 Mean':f11mean,'F1 stdev':f11stdev,
             }).to_csv('TempResults_Aug2022_BestModels5fold_Aggregate_BinaryModels1_All.csv')

In [16]:
pd.DataFrame({'Model':modelsagg,'Precision 2 Mean':prec2mean,'Precision 2 Stdev':prec2stdev,
              'Recall 2 Mean ':recall2mean,'Recall 2 Stdev ':recall2stdev,'F1 2 Mean':f12mean,'F1 2 stdev':f12stdev,
             }).to_csv('TempResults_Aug2022_BestModels5fold_Aggregate_BinaryModels2_All.csv')

In [17]:
pd.DataFrame({'Model':modelsagg,'Precision 3 Mean':prec3mean,'Precision 3 Stdev':prec3stdev,
              'Recall 1 Mean ':recall3mean,'Recall 3 Stdev ':recall3stdev,'F1 3 Mean':f13mean,'F1 stdev':f13stdev,
             }).to_csv('TempResults_Aug2022_BestModels5fold_Aggregate_BinaryModels3_All.csv')

In [18]:
df_tempresults = pd.read_csv('TempResults_Dec2022_BestModels5fold_Aggregate_BasicStats_All.csv')
list_inds = [0 for l in range(0,len(df_tempresults))]
print(list_inds)
modelname_nodir = []
inds = 0

# Get model name without direction
for i, row in df_tempresults.iterrows():
    print(row['Model'])
    list_inds[i] = inds
    # remove 'left' or 'right ' and find the match
    str_model = row['Model'].replace('_left','')
    str_model = str_model.replace('_right','')
    modelname_nodir.append(str_model)
df_tempresults['Model_Nodir'] = modelname_nodir
print(df_tempresults)

modelnames_unique = (list(set(modelname_nodir)))
print(modelnames_unique)

# Aggregate over models with same nodir name
prec_mean = []
prec_stdev = []
recall_mean = []
recall_stdev =[]
f1_mean = []
f1_stdev = []
mae_mean = []
mae_stdev = []
for i in modelnames_unique:
    rows = df_tempresults[df_tempresults['Model_Nodir']==i]
    print(rows)
    print('-'*80)
    newmeanprec = rows['Precision Mean'].mean(axis=0)
    prec_mean.append(newmeanprec)
    newstdevprec = rows['Precision Stdev'].mean(axis=0)
    prec_stdev.append(newstdevprec)
    
    newmeanrecall = rows['Recall Mean '].mean(axis=0)
    recall_mean.append(newmeanrecall)
    newstdevrecall = rows['Recall Stdev '].mean(axis=0)
    recall_stdev.append(newstdevrecall)
    
    newmeanf1 = rows['F1 Mean'].mean(axis=0)
    f1_mean.append(newmeanf1)
    newstdevf1 = rows['F1 stdev'].mean(axis=0)
    f1_stdev.append(newstdevf1)
    
    newmeanmae = rows['MAE Mean '].mean(axis=0)
    mae_mean.append(newmeanmae)
    newstdevmae = rows['MAE Stdev '].mean(axis=0)
    mae_stdev.append(newstdevmae)
    

[0, 0]
model_RESNET_left_spine_double
model_RESNET_right_spine_double
   Unnamed: 0                            Model  Precision Mean  \
0           0   model_RESNET_left_spine_double       63.608440   
1           1  model_RESNET_right_spine_double       65.055699   

   Precision Stdev  Recall Mean   Recall Stdev    F1 Mean   F1 stdev  \
0         1.242874      1.201356      63.611837  1.156649  63.235987   
1         1.182215      0.789779      65.129507  1.230717  64.582605   

   MAE Mean   MAE Stdev                 Model_Nodir  
0   0.427250    0.015804  model_RESNET_spine_double  
1   0.402707    0.015661  model_RESNET_spine_double  
['model_RESNET_spine_double']
   Unnamed: 0                            Model  Precision Mean  \
0           0   model_RESNET_left_spine_double       63.608440   
1           1  model_RESNET_right_spine_double       65.055699   

   Precision Stdev  Recall Mean   Recall Stdev    F1 Mean   F1 stdev  \
0         1.242874      1.201356      63.611837  1.

In [19]:
pd.DataFrame({'Model':modelnames_unique,'Precision Mean':prec_mean,'Precision Stdev':prec_stdev,
              'Recall Mean ':recall_mean,'Recall Stdev ':recall_stdev,'F1 1 Mean':f1_mean,'F1 stdev':f1_stdev,
              'MAE Mean':mae_mean,'MAE stdev':mae_stdev}).to_csv('TempResults_Aug2022_BestModels5fold_Aggregated_Sides_All.csv')